## Time spent
- 12/7:

## Outline

- Directed Graph
- Directed Graph: Ordered
- Directed Graph: Ordered, Competitive

Methods
- Build nodes
- Add edges according to rules
- Calculate component sizes

# To-do
- Implement DER
- Implement ODER, C-ODER
- Visualization..?
- Calculate component sizes of a graph with Tarjan algorithm
- Plot of largest SCC

Go though C++ tutorials linked to in Glotzdocs.

Speed up options:
- Multiprocessing and pool
- Cython

In [56]:
# Import needed packages
import time
import itertools
import numpy as np
import pandas as pd
import scipy
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# not yet installed
import networkx as nx
import percolate

In [3]:
# For cleanliness, ignore warnings after first appearance. 
import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')

In [22]:
from sem_graphs import gnp_random_graph

# 00 - Percolation implementations (simple)

### Network science background
- Erdos-Renyi graphs (random, non-directed graphs): [link here](https://en.wikipedia.org/wiki/Erd%C5%91s%E2%80%93R%C3%A9nyi_model)
- Directed Erdos-Renyi graphs, DER (directed edges)
- Paper covers two models:
  - Ordered, directed Erdos-Renyis (ODER)
  - Competitive ODER
  
I'm currently using Networkx to play around with these. In the generator source code [here](https://github.com/networkx/networkx/blob/1174e443263f8a60dc82083ad1c563a4c25e5582/networkx/generators/random_graphs.py), `erdos_renyi_graph` is just an alias for `gnp_random_graph`.

Algorithm for ODER:
- begin with a set of $n$ isolated nodes on which we have placed an arbitrarry ordering from 1 to $n$
- at each step, add a single directed edge between two nodes selected uniformly at random
    - the head of the directed edge = node higher in ordering
    - ...UNLESS the edge already exists in the graph
- repeat steps until $m$ edges have been added (edges and reverse edges count as separate edges)

In [52]:
def check_if_edge_exists(proposed,edges):
    for item in edges:
        if proposed==item: return True
        else: pass

In [75]:
# ODER implementation
def ODER(n,m):
    # set up list nodes with ranked order
    n_list = np.linspace(0,n-1,n).astype(int)

    # add m edges to the plot
    edge_list = []
    while len(edge_list)<m:
        first_node, second_node = random.sample(list(n_list),2)
        proposed_edge = tuple(sorted((first_node,second_node)))
        if check_if_edge_exists(proposed_edge,edge_list):
            if check_if_edge_exists(tuple(reversed(proposed_edge)),edge_list): pass
            else: edge_list.append(tuple(reversed(proposed_edge)))
        else: edge_list.append(proposed_edge)
    return edge_list

In [80]:
# this can go up to 10^4 okay
n = 10
m = 20
test = ODER(n,m)

[(4, 7), (1, 8), (2, 9), (3, 4), (1, 6), (0, 6), (0, 9), (2, 4), (0, 4), (8, 1), (3, 8), (6, 7), (5, 8), (1, 2), (2, 3), (2, 1), (9, 2), (2, 8), (0, 8), (9, 0)]


In [152]:
# C-ODER implementation
def CODER(n,m):
    # set up list nodes with ranked order
    n_list = np.linspace(0,n-1,n).astype(int)
    
    # add m edges to the plot
    edge_list = []
    while len(edge_list)<m:
        # samples without replacement (no duplicates)
        first_node, second_node, third_node = random.sample(list(n_list),3)
        proposed_edges = tuple(itertools.combinations(tuple(sorted((first_node,second_node,third_node))), 2))
        
        # for nodes with min difference, check if they/their reverse already exist; if no, add them
        difference = np.asarray([nodes[1]-nodes[0] for nodes in proposed_edges])
        idx = np.where(difference == difference.min())[0]
        for i in idx:
            proposed_edge = proposed_edges[i]
            # avoid duplicates; adds in reverse edges
            if check_if_edge_exists(proposed_edge,edge_list):
                if check_if_edge_exists(tuple(reversed(proposed_edge)),edge_list): pass
                else: edge_list.append(tuple(reversed(proposed_edge)))
            else: edge_list.append(proposed_edge)
    return edge_list    

In [157]:
# This can also go up to 5e3 m okay-- 5e4 still didn't finish during a bathroom break
n = 10**2
m = 5e3
edge_density= m/n
print('Edge density: %s' %edge_density)
test = CODER(n,m)

Edge density: 50.0


For an edge density of 50 on a 10^6 node network, as in the paper, this would require 50*10^6 edges (50 edges on each node).

In [1]:
# Trying to use networkx
'''
n: number of nodes
p: probability of an edge being accepted
'''

def gnp_random_graph(n, p):
    G = nx.DiGraph()
    G.add_nodes_from(range(n))
    if p <= 0:
        return G
    if p >= 1:
        return complete_graph(n, create_using=G)

    # Itertools makes all possible edge permutations between nodes-- so size of edges is n(n-1)-- I think
    edges = itertools.permutations(range(n), 2)

    # Here is where I need to add competition/ 
    for e in edges:
        if random.random() < p:
            G.add_edge(*e)
    return G

## Playing around with networkx capabilities

So, looks like this is not so straightforward.

Networkx works from different graph classes, e.g. [here](https://github.com/networkx/networkx/blob/a660d5728b3b3463b08c03ab8138f62468487c71/networkx/classes/digraph.py).

Class documentation is here: https://docs.python.org/3/tutorial/classes.html

Read through this: http://www.souravsengupta.com/cds2015/python/LPTHW.pdf

Unit tests: http://docs.python-guide.org/en/latest/writing/tests/

In [23]:
# ER example here (done with networkx)
# Based on this, will take about 500 seconds to make a graph with 10e6 nodes
# Can definitely parallelize this with map/pool!
# Reference: http://chriskiehl.com/article/parallelism-in-one-line/ < this is python 2.7
# Good reference on python 3: https://www.ploggingdev.com/2017/01/multiprocessing-and-multithreading-in-python-3/

n = [5000]
timed = []
for N in n:
    start = time.time()
    gnp_random_graph(int(N),0.5)
    timed.append(time.time()-start)
    start = time.time()
#     er = nx.erdos_renyi_graph(int(N),.5)
    er = nx.erdos_renyi_graph(int(N),.5)
    timed.append(time.time()-start)
    
print(timed)
    
# plt.scatter(n,timed)
# plt.subplot(121)
# nx.draw(er, with_labels=True, font_weight='bold')
# plt.subplot(122)
# nx.draw_shell(er, with_labels=True, font_weight='bold')

TypeError: add_edge() argument after * must be an iterable, not int

In [ ]:
# DER example here
er = nx.erdos_renyi_graph(10,.5,directed=True)
plt.subplot(121)
nx.draw(er, with_labels=True, font_weight='bold')
plt.subplot(122)
nx.draw_shell(er, with_labels=True, font_weight='bold')

### Model 1: Ordered, Directed Erdos-Renyi (ODER)
- Generalization of the directed ER model to ordered graph
- Form two large components, which explosively merge (discontinuous jump in the size of the largest strongly connected component)

Ordered, directed graphs can be made using the OrderedDiGraph class, source [here](https://github.com/networkx/networkx/blob/386b71a7af6c4898331f62987d8ced3f5621b680/networkx/classes/ordered.py).

From the python `collections` documentation [here](https://docs.python.org/3/library/collections.html#collections.OrderedDict): "an OrderedDict is a dict that remembers the order that keys were first inserted. If a new entry overwrites an existing entry, the original insertion position is left unchanged. Deleting an entry and reinserting it will move it to the end."

In [ ]:
# ODER example here

### Model 2: Competitive ODER
- Adds competition: preference for connecting nodes of similar rank
- See similar discontinuous jump in cluster size, but "more explosive"
- Get an effective phase separation of the two large components: one containing the lower-ranked users, one containing the higher-ranked users
- TAKEAWAY: Some bias towards grouping similar-ranked nodes leads to formation of two distinct groups of nodes (classes) with little flow of information between the classes

In [ ]:
# CODER example here

# 01 - Clustering implementations

### Classical non-directed NZ clustering algorithm

In [ ]:
# Newmann-Ziff implementation on ER

### Clustering algorithm implemented in paper
Implementation of Tarjan algorithm. Python source [here](https://github.com/bwesterb/py-tarjan).

Gives $O(E\log{E})$ clustering performance when using the pseudocode in section 5.

In [ ]:
# Implementation of clustering algorithm from the paper

# 02 - Thermodynamics

In [ ]:
# Plot SCC ("strongly connected component") versus edge density, per paper

In [ ]:
# Find critical exponents-- they actually don't have them

# 03 - Something else interesting?
- Can we say something else interesting from this work that the authors might not have thought about?
- What could have made this paper more interesting?
- How innovative actually is this?